# Contributors : 
- Boris Bobylkov - 21317097
- Italo Terto da Silva - 21326312
- Szymon Szulc - 21323208

# Game suggestions :
- [Breakout](https://www.gymlibrary.dev/environments/atari/breakout/)
- [Pong](https://ale.farama.org/multi-agent-environments/pong/)

# Objectives
To implement a Reinforcement Learning (RL) agent using a Deep Q Network (DQN) applied to the one of the environments in OpenAI Gym for an Atari game. 
This assignment is marked out of 30 marks and includes extra credit worth 5 marks.

# Submission
- 1. Why Reinforcement Learning is the machine learning paradigm of choice for this task (3 marks)
- 2. The Gym Environment (3 marks):
- 3. Implementation (15 marks):
    - a. Capture and pre-processing of the data (3 marks).
    - b. The network structure (4 marks).
    - c. A very clear and extremely detailed discussion on the code where the Q learning update applied to the weights (3 marks).
    - For example: what is the update, how is the error calculated?
    - d. Independently researched concepts such as random seed initialisation, the impact of
    regularisers on scores, and techniques to counter catastrophic forgetting and maximisation
    bias (5 marks). An attempt must be made to code and evaluate the efficacy of one of these
    concepts.

Page 2 

Coding fragments and/or diagrams should be included to illustrate the concepts under discussion.

- 4. Plots (3 marks)
- 5. Videos and plots with short accompanying explanations of the information conveyed (1 marks).
- 6. Evaluation of the results (5 marks):
    - a. How does one evaluate the performance of the RL agent?
    - b. Are the metrics that we have seen to date relevant?

- 7. References (P/F).

# **CS4187 - Assignment 3**

## **Students**

Boris Bobylkov - *21317097*

Italo Terto da Silva - *21326312*

Symon Szulc - *21323208*

Code compiles with no errors.

### **Resources:**

*   List item
*   List item



## 1. **Why Reinforcement Learning is the machine learning paradigm of choice for this task?**

The Reinforcement Learning was the chosen ML paradigm for this task because

## **2. The Gym Environment**

### If using conda you need these :
conda create --name pong_env python=3.9 --yes

conda install tensorflow=2.10 --yes

pip install gymnasium ale-py AutoROM

AutoROM --accept-license

In [1]:
import gymnasium as gym
import ale_py

# Register ALE environments
gym.register_envs(ale_py)

# Create the Assault environment
env = gym.make('ALE/Assault-v5', render_mode='human')

# Parameters
max_episodes = 10  # Stop after 10 episodes
episode_count = 0

# Run the game
while episode_count < max_episodes:
    observation, info = env.reset()
    done = False
    truncated = False
    episode_count += 1

    print(f"Starting episode {episode_count}...")
    while not done and not truncated:
        # Take random actions
        action = env.action_space.sample()
        observation, reward, done, truncated, info = env.step(action)
        print(f"Reward: {reward}, Done: {done}, Truncated: {truncated}")

    print(f"Episode {episode_count} ended.\n")

print("Reached the maximum number of episodes. Exiting.")
env.close()


A.L.E: Arcade Learning Environment (version 0.10.1+6a7e0ae)
[Powered by Stella]


Starting episode 1...
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 21.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Truncated: False
Reward: 0.0, Done: False, Trunc

## **3. Implementation**

### Importing Libraries

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.optimizers import Adam

### **a. Capture and pre-processing of the data**

In [3]:
print("Capture and Pre-Processed")

Capture and Pre-Processed


### **b. The Network Structure**

In [4]:
class DQN():
    def __init__(self, input_shape, n_actions, lerning_rate=0.0001, loss='mse'):
        # Setting the parameters to use in the model
        self.input_shape = input_shape
        self.n_actions = n_actions
        self.lerning_rate = lerning_rate
        self.loss = loss

        # Creating the model
        self.model = self.create_model()

    def create_model(self):
        # Creating a sequential model
        model = Sequential()

        # Adding the layers to the model
        model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=self.input_shape))
        model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
        model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.n_actions))

        # Compiling the model
        model.compile(optimizer=Adam(learning_rate=self.lerning_rate), loss=self.loss)

        return model

## **4. Plots:**

## **5. Videos and plots with short accompanying explanations of the information conveyed**

Also no idea about this

## **6. Evaluation of the results**

a. How does one evaluate the performance of the RL agent?

b. Are the metrics that we have seen to date relevant?